In [1]:
versioninfo()

Julia Version 1.5.1
Commit 697e782ab8 (2020-08-25 20:08 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, skylake)


In [ ]:
]activate .

In [3]:
]instantiate

In [4]:
using Squareness

## Simple Newton Method

In [5]:
function _issquare_with_simple_newton(x::I) where {I <: Integer}
    x == 0 && return true
    xtz = trailing_zeros(x)
    isodd(xtz) && return false
    x >>= unsigned(xtz)
    o_xrt::I = x
    xrt::I = (x + 0x1) >> 0x1
    xrt == 0 && return false  # overflow
    while xrt < o_xrt
        o_xrt = xrt
        xrt = (xrt + x ÷ xrt) >> 0x1
    end
    xrt * xrt == x
end

issquare_with_simple_newton(x::Signed) = x < 0 ? false : _issquare_with_simple_newton(x % Unsigned)
issquare_with_simple_newton(x::Unsigned) = _issquare_with_simple_newton(x)
issquare_with_simple_newton(x::BigInt) = x < 0 ? false : _issquare_with_simple_newton(x)

issquare_with_simple_newton (generic function with 3 methods)

### Simple Check

In [6]:
all(issquare(x) == issquare_with_simple_newton(x) == Squareness.issquare_with_isqrt(x) for x in rand(Int, 10000))

true

In [7]:
all(issquare(x) == issquare_with_simple_newton(x) == Squareness.issquare_with_isqrt(x) for x in rand(Int32, 10000))

true

## Benchmark

In [8]:
using BenchmarkTools

### Compare `issquare()`(with P-adic Newton) vs `issquare_with_isqrt()` vs `issquare_with_simple_newton()`

#### `Int64`

In [9]:
rand_int64() = rand(Int64) & typemax(Int64)

rand_int64 (generic function with 1 method)

In [10]:
@benchmark issquare(n) setup=(n=rand_int64())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.882 ns (0.00% GC)
  median time:      5.221 ns (0.00% GC)
  mean time:        4.729 ns (0.00% GC)
  maximum time:     40.874 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

In [11]:
@benchmark Squareness.issquare_with_isqrt(n) setup=(n=rand_int64())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     12.670 ns (0.00% GC)
  median time:      12.996 ns (0.00% GC)
  mean time:        13.183 ns (0.00% GC)
  maximum time:     54.264 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     999

In [12]:
@benchmark issquare_with_simple_newton(n) setup=(n=rand_int64())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.402 ns (0.00% GC)
  median time:      328.616 ns (0.00% GC)
  mean time:        230.272 ns (0.00% GC)
  maximum time:     642.168 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

#### `Int64` × 100

In [13]:
rand_int64(N) = rand(Int64, N) .& typemax(Int64)

rand_int64 (generic function with 2 methods)

In [14]:
@benchmark issquare.(n) setup=(n=rand_int64(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     484.974 ns (0.00% GC)
  median time:      584.278 ns (0.00% GC)
  mean time:        602.048 ns (1.13% GC)
  maximum time:     14.407 μs (94.24% GC)
  --------------
  samples:          10000
  evals/sample:     194

In [15]:
@benchmark Squareness.issquare_with_isqrt.(n) setup=(n=rand_int64(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     1.453 μs (0.00% GC)
  median time:      1.514 μs (0.00% GC)
  mean time:        1.542 μs (0.00% GC)
  maximum time:     5.389 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     10

In [16]:
@benchmark issquare_with_simple_newton.(n) setup=(n=rand_int64(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     17.051 μs (0.00% GC)
  median time:      23.412 μs (0.00% GC)
  mean time:        23.782 μs (0.00% GC)
  maximum time:     75.220 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

#### `Int32`

In [17]:
rand_int32() = rand(Int32) & typemax(Int32)

rand_int32 (generic function with 1 method)

In [18]:
@benchmark issquare(n) setup=(n=rand_int32())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.709 ns (0.00% GC)
  median time:      4.318 ns (0.00% GC)
  mean time:        3.820 ns (0.00% GC)
  maximum time:     34.003 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

In [19]:
@benchmark Squareness.issquare_with_isqrt(n) setup=(n=rand_int32())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.536 ns (0.00% GC)
  median time:      7.640 ns (0.00% GC)
  mean time:        7.896 ns (0.00% GC)
  maximum time:     38.855 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     999

In [20]:
@benchmark issquare_with_simple_newton(n) setup=(n=rand_int32())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     3.116 ns (0.00% GC)
  median time:      101.114 ns (0.00% GC)
  mean time:        73.857 ns (0.00% GC)
  maximum time:     260.022 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

#### `Int32` × 100

In [21]:
rand_int32(N) = rand(Int32, N) .& typemax(Int32)

rand_int32 (generic function with 2 methods)

In [22]:
@benchmark issquare.(n) setup=(n=rand_int32(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     445.763 ns (0.00% GC)
  median time:      532.295 ns (0.00% GC)
  mean time:        553.374 ns (1.29% GC)
  maximum time:     15.029 μs (96.17% GC)
  --------------
  samples:          10000
  evals/sample:     198

In [23]:
@benchmark Squareness.issquare_with_isqrt.(n) setup=(n=rand_int32(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     379.786 ns (0.00% GC)
  median time:      402.841 ns (0.00% GC)
  mean time:        423.361 ns (1.68% GC)
  maximum time:     15.248 μs (97.16% GC)
  --------------
  samples:          10000
  evals/sample:     201

In [24]:
@benchmark issquare_with_simple_newton.(n) setup=(n=rand_int32(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     5.588 μs (0.00% GC)
  median time:      8.019 μs (0.00% GC)
  mean time:        8.207 μs (0.00% GC)
  maximum time:     21.728 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     6

#### `Int16`

In [25]:
rand_int16() = rand(Int16) & typemax(Int16)

rand_int16 (generic function with 1 method)

In [26]:
@benchmark issquare(n) setup=(n=rand_int16())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.954 ns (0.00% GC)
  median time:      9.816 ns (0.00% GC)
  mean time:        7.687 ns (0.00% GC)
  maximum time:     42.653 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

In [27]:
@benchmark Squareness.issquare_with_isqrt(n) setup=(n=rand_int16())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.485 ns (0.00% GC)
  median time:      7.647 ns (0.00% GC)
  mean time:        7.688 ns (0.00% GC)
  maximum time:     37.499 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     999

In [28]:
@benchmark issquare_with_simple_newton(n) setup=(n=rand_int16())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.590 ns (0.00% GC)
  median time:      52.829 ns (0.00% GC)
  mean time:        39.345 ns (0.00% GC)
  maximum time:     150.818 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

#### `Int16` × 100

In [29]:
rand_int16(N) = rand(Int16, N) .& typemax(Int16)

rand_int16 (generic function with 2 methods)

In [30]:
@benchmark issquare.(n) setup=(n=rand_int16(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     392.302 ns (0.00% GC)
  median time:      464.297 ns (0.00% GC)
  mean time:        489.867 ns (1.53% GC)
  maximum time:     16.567 μs (95.63% GC)
  --------------
  samples:          10000
  evals/sample:     202

In [31]:
@benchmark Squareness.issquare_with_isqrt.(n) setup=(n=rand_int16(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     391.194 ns (0.00% GC)
  median time:      411.816 ns (0.00% GC)
  mean time:        433.112 ns (1.80% GC)
  maximum time:     18.438 μs (97.60% GC)
  --------------
  samples:          10000
  evals/sample:     201

In [32]:
@benchmark issquare_with_simple_newton.(n) setup=(n=rand_int16(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     2.584 μs (0.00% GC)
  median time:      3.437 μs (0.00% GC)
  mean time:        3.472 μs (0.00% GC)
  maximum time:     9.935 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     9

#### `Int128`

In [33]:
rand_int128() = rand(Int128) & typemax(Int128)

rand_int128 (generic function with 1 method)

In [34]:
@benchmark issquare(n) setup=(n=rand_int128())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.708 ns (0.00% GC)
  median time:      25.444 ns (0.00% GC)
  mean time:        18.498 ns (0.00% GC)
  maximum time:     93.586 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

In [35]:
@benchmark Squareness.issquare_with_isqrt(n) setup=(n=rand_int128())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     43.527 ns (0.00% GC)
  median time:      45.331 ns (0.00% GC)
  mean time:        46.067 ns (0.00% GC)
  maximum time:     126.147 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     990

In [36]:
@benchmark issquare_with_simple_newton(n) setup=(n=rand_int128())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.585 ns (0.00% GC)
  median time:      1.838 μs (0.00% GC)
  mean time:        1.253 μs (0.00% GC)
  maximum time:     2.446 μs (0.00% GC)
  --------------
  samples:          3991
  evals/sample:     1000

#### `Int128` × 100

In [37]:
rand_int128(N) = rand(Int128, N) .& typemax(Int128)

rand_int128 (generic function with 2 methods)

In [38]:
@benchmark issquare.(n) setup=(n=rand_int128(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     1.509 μs (0.00% GC)
  median time:      2.002 μs (0.00% GC)
  mean time:        2.053 μs (0.83% GC)
  maximum time:     173.143 μs (98.70% GC)
  --------------
  samples:          10000
  evals/sample:     10

In [39]:
@benchmark Squareness.issquare_with_isqrt.(n) setup=(n=rand_int128(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     4.560 μs (0.00% GC)
  median time:      4.618 μs (0.00% GC)
  mean time:        4.772 μs (0.00% GC)
  maximum time:     16.818 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     7

In [40]:
@benchmark issquare_with_simple_newton.(n) setup=(n=rand_int128(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     95.457 μs (0.00% GC)
  median time:      129.455 μs (0.00% GC)
  mean time:        131.133 μs (0.00% GC)
  maximum time:     287.884 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

#### `BigInt`

In [41]:
rand_mpz() = (big(rand(UInt128)) << rand(0:128)) + rand(UInt128)

rand_mpz (generic function with 1 method)

In [42]:
# @benchmark issquare(n) setup=(n=rand_mpz())
@benchmark Squareness.issquare_with_padic_newton(n) setup=(n=rand_mpz())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.553 ns (0.00% GC)
  median time:      293.926 μs (1.25% GC)
  mean time:        268.913 μs (3.09% GC)
  maximum time:     630.357 μs (1.34% GC)
  --------------
  samples:          20
  evals/sample:     999

In [43]:
@benchmark Squareness.issquare_with_isqrt(n) setup=(n=rand_mpz())

BenchmarkTools.Trial: 
  memory estimate:  88 bytes
  allocs estimate:  5
  --------------
  minimum time:     133.862 ns (0.00% GC)
  median time:      244.319 ns (0.00% GC)
  mean time:        395.420 ns (20.99% GC)
  maximum time:     173.369 μs (75.62% GC)
  --------------
  samples:          10000
  evals/sample:     595

In [44]:
@benchmark issquare_with_simple_newton(n) setup=(n=rand_mpz())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     8.577 ns (0.00% GC)
  median time:      20.595 μs (0.00% GC)
  mean time:        29.174 μs (23.94% GC)
  maximum time:     139.787 μs (55.07% GC)
  --------------
  samples:          174
  evals/sample:     999